Importing essential functions.

In [1]:
# from navicat_mikimo.plot_function import plot_2d_combo, plot_3d_contour_regions_np
import numpy as np
import h5py
import seaborn as sns
import matplotlib.collections as mcoll
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib
from matplotlib import cm
from matplotlib.ticker import FuncFormatter
from navicat_volcanic.helpers import bround
from navicat_volcanic.plotting2d import beautify_ax

sns.set()
sns.set_palette("colorblind", 8)
sns.set_style("ticks", {"xtick.major.size": 18, "ytick.major.size": 18})
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
# |
from  matplotlib.colors import LinearSegmentedColormap
c = ["#6E0000", "#FF2600","lightcoral","white", "palegreen","#009051","#00753A"]
v = [0,.2,.35,.5,.65,.8,1.]
l = list(zip(v,c[::-1]))
rwg=LinearSegmentedColormap.from_list('rg',l, N=1024)

/home/pregabalin/anaconda3/envs/suisei/lib/python3.9/site-packages/numpy/core/getlimits.py:542: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
def plot_3d_np(
    xint,
    yint,
    grid,
    ymin,
    ymax,
    x1min,
    x1max,
    x2min,
    x2max,
    x1base,
    x2base,
    x1label="X1-axis",
    x2label="X2-axis",
    ylabel="Y-axis",
    filename="plot.png",
    cmap="seismic",
    figsize=[7.5, 4],
):
    """Like plot_3d_m but without points, for the mkm maps (t,T) or ([t,T], descriptor))"""
    fig, ax = plt.subplots(
        frameon=False, figsize=figsize, dpi=300, constrained_layout=True
    )
    grid = np.clip(grid, ymin, ymax)
    norm = cm.colors.Normalize(vmax=ymax, vmin=ymin)
    ax = beautify_ax(ax)

    increment = np.round((ymax - ymin) / 10, 1)
    levels = np.arange(ymin, ymax + increment, increment / 100)

    cset = ax.contourf(
        xint,
        yint,
        grid,
        levels=levels,
        norm=norm,
        cmap=cm.get_cmap(cmap, len(levels)),
        vmin=ymin,
        vmax=ymax,
    )

    xticks = np.arange(x1min, x1max + 0.1, x1base)
    yticks = np.arange(x2min, x2max + 0.1, x2base)
    plt.xlabel(x1label)
    plt.ylabel(x2label)
    plt.xlim(np.min(xticks), np.max(xticks))
    plt.ylim(np.min(yticks), np.max(yticks))
    plt.xticks(xticks)
    plt.yticks(yticks)

    
    def fmt(x, pos):
        return "%.0f" % x

    cbar = fig.colorbar(cset, format=FuncFormatter(fmt))
    cbar.set_label(ylabel, labelpad=3)

    # tick_labels = ['{:.2f}'.format(value) for value in levels]
    tick_positions = np.arange(ymin, ymax + 0.1, increment)
    tick_labels = [f"{value:.1f}" for value in tick_positions]

    cbar.set_ticks(tick_positions)
    cbar.set_ticklabels(tick_labels)
    cbar.ax.tick_params(pad=35)
    cbar.ax.set_ylim(ymin, ymax)
    plt.setp(cbar.ax.get_yticklabels(), ha="right")

    plt.savefig(filename)
    # plt.tight_layout()
    # plt.show()

def plot_2d_combo(
    x,
    y,
    px,
    py,
    ci=None,
    xmin=0,
    xmax=100,
    xbase=20,
    ybase=10,
    xlabel="X-axis",
    ylabel="Y-axis",
    filename="plot.png",
    rid=None,
    ms=None,
    rb=None,
    plotmode=1,
    labels=None,
    figsize=[7.5, 4],
):
    COLORS = [
        "#FF2600",
        "#009051",
        "#4169E1",
        "#9607BC",
        "#EA2CD3",
        "#D88918",
        "#148873",
        "#000000",
    ]

    marker = ["o", "^", "s", "P", "*", "X", "d", "D"]
    fig, ax = plt.subplots(
        frameon=False, figsize=figsize, dpi=300, constrained_layout=True
    )
    # Labels and key
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(xmin, xmax)
    plt.xticks(np.arange(xmin, xmax + 0.1, xbase))

    # no scatter plot
    if plotmode == 0:
        for i, yi in enumerate(y):
            ax.plot(
                x, yi, "-", linewidth=2.2, color=COLORS[i], alpha=0.95, label=labels[i]
            )
            ax = beautify_ax(ax)
            if rid is not None and rb is not None:
                avgs = []
                rb.append(xmax)
                for i in range(len(rb) - 1):
                    avgs.append((rb[i] + rb[i + 1]) / 2)
                for i in rb:
                    ax.axvline(
                        i, linestyle="dashed", color="black", linewidth=2, alpha=0.75
                    )

    # mono color scatter plot
    elif plotmode > 0:
        for i, yi in enumerate(y):
            ax.plot(
                x,
                yi,
                "-",
                linewidth=2,
                color=COLORS[i],
                alpha=0.95,
                zorder=1,
                label=labels[i],
            )
            ax = beautify_ax(ax)
            if rid is not None and rb is not None:
                avgs = []
                rb.append(xmax)
                for i in range(len(rb) - 1):
                    avgs.append((rb[i] + rb[i + 1]) / 2)
                for i in rb:
                    ax.axvline(
                        i,
                        linestyle="dashed",
                        color="black",
                        linewidth=2,
                        alpha=0.75,
                        zorder=3,
                    )
            if ci[i] is not None:
                plot_ci(ci[i], x, y[i], ax=ax)
            plotpoints(ax, px, py[i], np.repeat([COLORS[i]], len(px)), ms, plotmode)

    ymin, ymax = ax.get_ylim()
    ymax = bround(ymax, ybase, type="max")
    ymin = bround(ymin, ybase, type="min")
    plt.ylim(0, ymax)
    plt.yticks(np.arange(0, ymax + 0.1, ybase))
    plt.legend(fontsize=16, loc="upper right", frameon=False, borderpad=0)

    fig.savefig(filename, dpi=400)
#

### Figure 4. Rh-catalyzed hydroformylation

- Catalyst-temperature map

In [20]:
filename = "data/hf/data_tv_t.h5"
with h5py.File(filename, "r") as f:
    group = f["data"]
    xint = group["xint"][:]
    t_points = group["t_points"][:]
    grid_d = group["grid"][:]
    tag = group["tag"][:]
    x1label = group["x1label"][:]
    x2label = group["x2label"][:]

# ------------------------------------------------------------

tag = "INT4L"
x1label = "$ΔG_{RRS}$" + f"({tag}) [kcal/mol]"
x2label = "Temperature [K]"
x1base = 10
x2base = 20

sfilename = "HaT_selectivity_map_hf.png"
prod = ["L", "B"]
slabel = "$log_{10}$" + f"({prod[0]}/{prod[1]})"
min_ratio = -2
max_ratio = 2
selectivity_ratio = np.log10(grid_d[0] / grid_d[1])
selectivity_ratio_ = np.clip(selectivity_ratio, min_ratio, max_ratio)
#selectivity_ratio_ = np.nan_to_num(selectivity_ratio_, nan=-2, posinf=2, neginf=-2)
from scipy.interpolate import griddata
grid = selectivity_ratio_
x, y = np.indices(grid.shape)
interp = np.array(grid)
interp[np.isnan(interp)] = griddata((x[~np.isnan(grid)], y[~np.isnan(grid)]), grid[~np.isnan(grid)],(x[np.isnan(grid)], y[np.isnan(grid)]))

plot_3d_np(
    xint,
    t_points,
    interp.T,
    -2,
    2,
    -30,
    20,
    280,
    400,
    x1base,
    x2base,
    x1label,
    x2label,
    slabel,
    sfilename,
    rwg,
)

plt.axhline(y = 298.15, xmin = -40, xmax = 40, color = 'black', linestyle = 'dashed') 
plt.axhline(y = 353.15, xmin = -40, xmax = 40, color = 'black', linestyle = 'dashed') 

/tmp/ipykernel_4991/1315971769.py:24: RuntimeWarning: divide by zero encountered in divide
  selectivity_ratio = np.log10(grid_d[0] / grid_d[1])
/tmp/ipykernel_4991/1315971769.py:24: RuntimeWarning: invalid value encountered in divide
  selectivity_ratio = np.log10(grid_d[0] / grid_d[1])
/tmp/ipykernel_4991/1315971769.py:24: RuntimeWarning: divide by zero encountered in log10
  selectivity_ratio = np.log10(grid_d[0] / grid_d[1])


/tmp/ipykernel_4991/18281058.py:37: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap=cm.get_cmap(cmap, len(levels)),


- Catalyst-time map

In [11]:
# read h5 files
filename = "data/hf/data_tv_time.h5"
with h5py.File(filename, "r") as f:
    group = f["data"]
    xint = group["xint"][:]
    t_points = group["t_points"][:]
    grid_d = group["grid"][:]
    tag = group["tag"][:]
    x1label = group["x1label"][:]
    x2label = group["x2label"][:]

# ------------------------------------------------------------

tag = "INT4L"
x1label = "$ΔG_{RRS}$" + f"({tag}) [kcal/mol]"
x2label = "log$_{10}$(time) [s]"
x1base = 10
x2base = 0.2

sfilename = "Hat_selectivity_map_hf.png"
prod = ["L", "B"]
slabel = "$log_{10}$" + f"({prod[0]}/{prod[1]})"
min_ratio = -2
max_ratio = 2
selectivity_ratio = np.log10(grid_d[0] / grid_d[1])
selectivity_ratio_ = np.clip(selectivity_ratio, min_ratio, max_ratio)
#selectivity_ratio_ = np.nan_to_num(selectivity_ratio_, nan=-3, posinf=3, neginf=-3)
from scipy.interpolate import griddata
grid = selectivity_ratio_
x, y = np.indices(grid.shape)
interp = np.array(grid)
interp[np.isnan(interp)] = griddata((x[~np.isnan(grid)], y[~np.isnan(grid)]), grid[~np.isnan(grid)],(x[np.isnan(grid)], y[np.isnan(grid)]))

ax2 = plot_3d_np(
    xint,
    t_points,
    selectivity_ratio_.T,
    -2,
    2,
    -30,
    20,
    t_points.min(),
    t_points.max(),
    x1base,
    x2base,
    x1label,
    x2label,
    slabel,
    sfilename,
    rwg,
)

plt.axhline(y = 4.94, xmin = -41, xmax = 41, color = 'black', linestyle = 'dashed') 
plt.axhline(y = 4.33, xmin = -41, xmax = 41, color = 'black', linestyle = 'dashed') 

/tmp/ipykernel_6259/1085030691.py:25: RuntimeWarning: invalid value encountered in divide
  selectivity_ratio = np.log10(grid_d[0] / grid_d[1])
/tmp/ipykernel_6259/1085030691.py:25: RuntimeWarning: divide by zero encountered in log10
  selectivity_ratio = np.log10(grid_d[0] / grid_d[1])
/tmp/ipykernel_6259/3932567192.py:109: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap=cm.get_cmap(cmap, len(levels)),


- 2D volcano plots

In [22]:
def plot_2d_combo(
    x,
    y,
    px,
    py,
    ci=None,
    xmin=0,
    xmax=100,
    xbase=20,
    ybase=10,
    xlabel="X-axis",
    ylabel="Y-axis",
    filename="plot.png",
    rid=None,
    ms=None,
    rb=None,
    plotmode=1,
    labels=None,
    figsize=[7.5, 4],
):
    COLORS = [
        "#FF2B00",
        "#017D48",
        "#4169E1",
        "#9607BC",
        "#EA2CD3",
        "#D88918",
        "#148873",
        "#000000",
    ]

    marker = ["o", "^", "s", "P", "*", "X", "d", "D"]
    fig, ax = plt.subplots(
        frameon=False, figsize=figsize, dpi=300, constrained_layout=True
    )
    # Labels and key
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(xmin, xmax)
    plt.xticks(np.arange(xmin, xmax + 0.1, xbase))

    # no scatter plot
    if plotmode == 0:
        for i, yi in enumerate(y):
            ax.plot(
                x, yi, "-", linewidth=2.2, color=COLORS[i], alpha=0.95, label=labels[i]
            )
            ax = beautify_ax(ax)
            if rid is not None and rb is not None:
                avgs = []
                rb.append(xmax)
                for i in range(len(rb) - 1):
                    avgs.append((rb[i] + rb[i + 1]) / 2)
                for i in rb:
                    ax.axvline(
                        i, linestyle="dashed", color="black", linewidth=2, alpha=0.75
                    )

    # mono color scatter plot
    elif plotmode > 0:
        for i, yi in enumerate(y):
            ax.plot(
                x,
                yi,
                "-",
                linewidth=2,
                color=COLORS[i],
                alpha=0.95,
                zorder=1,
                label=labels[i],
            )
            ax = beautify_ax(ax)
            if rid is not None and rb is not None:
                avgs = []
                rb.append(xmax)
                for i in range(len(rb) - 1):
                    avgs.append((rb[i] + rb[i + 1]) / 2)
                for i in rb:
                    ax.axvline(
                        i,
                        linestyle="dashed",
                        color="black",
                        linewidth=2,
                        alpha=0.75,
                        zorder=3,
                    )
            if ci[i] is not None:
                plot_ci(ci[i], x, y[i], ax=ax)
            plotpoints(ax, px, py[i], np.repeat([COLORS[i]], len(px)), ms, plotmode)

    ymin, ymax = ax.get_ylim()
    ymax = bround(ymax, ybase, type="max")
    ymin = bround(ymin, ybase, type="min")
    plt.ylim(0, 1.10)
    plt.yticks(np.arange(0, 1.01, ybase))
    #plt.legend(fontsize=16, loc="upper left", frameon=False, borderpad=0.1, handlelength=1)

    fig.savefig(filename, dpi=400)

In [23]:
filename = "data/hf/vp/data_b.h5"
# read h5 files
with h5py.File(filename, "r") as f:
    # access the group containing the datasets
    group = f["data"]
    # load each dataset into a numpy array
    descr_all = group["descr_all"][:]
    prod_conc_ = group["prod_conc_"][:]
    descrp_pt = group["descrp_pt"][:]
    prod_conc_pt_ = group["prod_conc_pt_"][:]
    cb = group["cb"][:]
    ms = group["ms"][:]
    cb = np.char.decode(cb)
    ms = np.char.decode(ms)
    tag = group["tag"][:]
    xlabel = group["xlabel"][:]
    ylabel = group["ylabel"][:]

tag = "INT4L"
xlabel = "$ΔG_{RRS}$" + f"({tag}) [kcal/mol]"
ylabel = ylabel[0].decode()

if np.any(np.max(prod_conc_pt_) > 10):
    print("Concentration likely reported as %yield, set y_base to 10")
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8)
    if ybase == 0:
        ybase = 5
    ylabel = "%yield"
else:
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8, 1)
    if ybase == 0:
        ybase = 0.05
    ylabel = "Final product conc. [M]"

xbase = np.round((np.max(descr_all) - np.min(descr_all)) / 8)
if xbase == 0:
    xbase = 5

product_id = ["L", "B"]

plot_2d_combo(
    descr_all,
    prod_conc_,
    descrp_pt,
    prod_conc_pt_,
    # xmin=descr_all[0],
    # xmax=descr_all[-1],
    xmin=-30,
    xmax=20,
    ms=ms,
    xbase=10,
    ybase=0.2,
    xlabel=xlabel,
    ylabel=ylabel,
    filename=f"km_volcano_{tag}_combo_polished_b.png",
    plotmode=0,
    labels=product_id,
    ci=[None] * 2,
)

#plt.axvline(x = -24.45, color = 'black', linestyle = 'dashed') 
plt.axvline(x = -15.35, color = 'black', linestyle = 'dashed') 
plt.axvline(x = -24.45, color = 'black', linestyle = 'dashed') 
plt.axvline(x = 7.13, color = 'black', linestyle = 'dashed') 

In [24]:
filename = "data/hf/vp/data_a.h5"
# read h5 files
with h5py.File(filename, "r") as f:
    # access the group containing the datasets
    group = f["data"]
    # load each dataset into a numpy array
    descr_all = group["descr_all"][:]
    prod_conc_ = group["prod_conc_"][:]
    descrp_pt = group["descrp_pt"][:]
    prod_conc_pt_ = group["prod_conc_pt_"][:]
    cb = group["cb"][:]
    ms = group["ms"][:]
    cb = np.char.decode(cb)
    ms = np.char.decode(ms)
    tag = group["tag"][:]
    xlabel = group["xlabel"][:]
    ylabel = group["ylabel"][:]

tag = "INT4L"
xlabel = "$ΔG_{RRS}$" + f"({tag}) [kcal/mol]"
ylabel = ylabel[0].decode()

if np.any(np.max(prod_conc_pt_) > 10):
    print("Concentration likely reported as %yield, set y_base to 10")
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8)
    if ybase == 0:
        ybase = 5
    ylabel = "%yield"
else:
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8, 1)
    if ybase == 0:
        ybase = 0.05
    ylabel = "Final product conc. [M]"

xbase = np.round((np.max(descr_all) - np.min(descr_all)) / 8)
if xbase == 0:
    xbase = 5

product_id = ["L", "B"]

plot_2d_combo(
    descr_all,
    prod_conc_,
    descrp_pt,
    prod_conc_pt_,
    # xmin=descr_all[0],
    # xmax=descr_all[-1],
    xmin=-30,
    xmax=20,
    ms=ms,
    xbase=10,
    ybase=0.2,
    xlabel=xlabel,
    ylabel=ylabel,
    filename=f"km_volcano_{tag}_combo_polished_b.png",
    plotmode=0,
    labels=product_id,
    ci=[None] * 2,
)
plt.axvline(x = -15.35, color = 'black', linestyle = 'dashed') 
plt.axvline(x = -24.45, color = 'black', linestyle = 'dashed') 
plt.axvline(x = 7.13, color = 'black', linestyle = 'dashed') 


### Figure 5. Replot activity and selectivity map with time and temperature as descriptors

- Catalytic hydrosilylation of CO$_2$ with metal pincer complexes

In [6]:
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

def plot_3d_contour_regions_np(
    xint,
    yint,
    grid,
    x1min,
    x1max,
    x2min,
    x2max,
    x1base,
    x2base,
    x1label="X1-axis",
    x2label="X2-axis",
    ylabel="Y-axis",
    filename="plot.png",
    nunique=2,
    figsize=[7.5, 4],
    id_labels=[],
):
    """Like plot_3d_np, but for when there are more than 2 products"""
    colors = [
        "#FF6347",
        "#009051",
        "#4169E1",
    ]
   
    #colors = [(255, 99, 71), (0, 144, 81), (65, 105, 225), (150, 7, 188)]
    fig, ax = plt.subplots(
        frameon=False, figsize=figsize, dpi=300, constrained_layout=True
    )
    ax = beautify_ax(ax)
    levels = np.arange(-0.1, nunique + 0.9, 1)
    cmap = ListedColormap(colors, )
    # cset = ax.contourf(
    #    xint, yint, grid, levels=levels, cmap=cm.get_cmap("Dark2", nunique + 1)
    # )
    cset = ax.contourf(xint, yint, grid, levels=levels, cmap=cmap)

    # Labels and key
    xticks = np.arange(x1min, x1max + 0.1, x1base)
    yticks = np.arange(x2min, x2max + 0.1, x2base)
    plt.xlabel(x1label)
    plt.ylabel(x2label)
    plt.xlim(np.min(xticks), np.max(xticks))
    plt.ylim(np.min(yticks), np.max(yticks))
    plt.xticks(np.arange(x1min, x1max + 0.1, x1base))
    plt.yticks(np.arange(x2min, x2max + 0.1, x2base))
    ax.contour(xint, yint, grid, cset.levels, colors="black", linewidths=0.1)

    def fmt(x, pos):
        return "%.0f" % x

    cbar = fig.colorbar(cset, format=FuncFormatter(fmt))
    cbar.set_ticks([])
    cbar.set_label(ylabel, labelpad=3)
    for j, tlab in enumerate(id_labels):
        cbar.ax.text(
            2,
            0.4 + j,
            tlab,
            ha="center",
            va="center",
            weight="light",
            fontsize=14,
            rotation=90,
        )
        cbar.ax.get_yaxis().labelpad = 37.5


    plt.savefig(filename)

def plot_3d_np(
    xint,
    yint,
    grid,
    ymin,
    ymax,
    x1min,
    x1max,
    x2min,
    x2max,
    x1base,
    x2base,
    x1label="X1-axis",
    x2label="X2-axis",
    ylabel="Y-axis",
    filename="plot.png",
    cmap="seismic",
    figsize=[7.5, 4],
):
    """Like plot_3d_m but without points, for the mkm maps (t,T) or ([t,T], descriptor))"""
    fig, ax = plt.subplots(
        frameon=False, figsize=figsize, dpi=300, constrained_layout=True
    )
    grid = np.clip(grid, ymin, ymax)
    norm = cm.colors.Normalize(vmax=ymax, vmin=ymin)
    ax = beautify_ax(ax)

    increment = np.round((ymax - ymin) / 10, 1)
    levels = np.arange(ymin, ymax + increment, increment / 100)

    cset = ax.contourf(
        xint,
        yint,
        grid,
        levels=levels,
        norm=norm,
        cmap=cm.get_cmap(cmap, len(levels)),
    )

    xticks = np.arange(x1min, x1max + 0.1, x1base)
    yticks = np.arange(x2min, x2max + 0.1, x2base)
    plt.xlabel(x1label)
    plt.ylabel(x2label)
    plt.xlim(np.min(xticks), np.max(xticks))
    plt.ylim(np.min(yticks), np.max(yticks))
    plt.xticks(xticks)
    plt.yticks(yticks)

    def fmt(x, pos):
        return "%.0f" % x

    cbar = fig.colorbar(cset, format=FuncFormatter(fmt))
    cbar.set_label(ylabel, labelpad=3)

    # tick_labels = ['{:.2f}'.format(value) for value in levels]
    tick_positions = np.arange(ymin, ymax + 0.1, increment)
    tick_labels = [f"{value:.1f}" for value in tick_positions]

    cbar.set_ticks(tick_positions)
    cbar.set_ticklabels(tick_labels)
    cbar.ax.tick_params(pad=35)
    cbar.ax.set_ylim(0, 1)
    plt.setp(cbar.ax.get_yticklabels(), ha="right")

    plt.savefig(filename)
    # plt.tight_layout()
    # plt.show()

In [26]:
# read h5 files
filename_activity = "data/pincer_co2/mkm_time_temperature_activity.h5"
with h5py.File(filename_activity, "r") as f:
    group = f["data"]
    temperatures_ = group["temperatures"][:]
    times_ = group["times"][:]
    agrid = group["agrid"][:]

filename_selectivity = "data/pincer_co2/mkm_time_temperature_selectivity.h5"
with h5py.File(filename_selectivity, "r") as f:
    group = f["data"]
    temperatures_ = group["temperatures"][:]
    times_ = group["times"][:]
    sgrid = group["dominant_indices"][:]

In [27]:
#013C13# plot

x1label = "Temperature [K]"
x2label = "log$_{10}$(time) [s]"
afilename = "Tt_activity_map.png"
sfilename = "Tt_selectivity_map.png"
alabel = "Total product conc. [M]"
slabel = "Major product"
x1base = 20
x2base = 0.5
product_id = ["HCOO[Si]", "CH$_2$(O[Si])$_2$", "CH$_3$(O[Si])"]


temp_lb = 280
temp_ub = 400
time_lb = 3
time_ub = 5

plot_3d_np(
    temperatures_,
    times_,
    agrid,
    0.0,
    1.00,
    temp_lb,
    temp_ub,
    time_lb,
    time_ub,
    x1base,
    x2base,
    x1label=x1label,
    x2label=x2label,
    ylabel=alabel,
    filename=afilename,
    cmap="jet",
)
plt.axhline(y = 4.63, xmin = -41, xmax = 41, color = 'black', linestyle = 'dashed') 
plt.axhline(y = 3.56, xmin = -41, xmax = 41, color = 'black', linestyle = 'dashed') 

plot_3d_contour_regions_np(
    temperatures_,
    times_,
    sgrid,
    temp_lb,
    temp_ub,
    time_lb,
    time_ub,
    x1base,
    x2base,
    x1label=x1label,
    x2label=x2label,
    ylabel=slabel,
    filename=sfilename,
    id_labels=product_id,
    nunique=len(product_id),
)
plt.axhline(y = 4.63, xmin = -41, xmax = 41, color = 'black', linestyle = 'dashed') 
plt.axhline(y = 3.56, xmin = -41, xmax = 41, color = 'black', linestyle = 'dashed') 

/tmp/ipykernel_4991/3932567192.py:109: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap=cm.get_cmap(cmap, len(levels)),


### Figure 6 Replot activity and selectivity map with Ha and temperature as descriptors

- Catalytic hydrosilylation of CO$_2$ with metal pincer complexes

In [3]:

# read h5 files 
filename = "data/pincer_co2/mkm_descr_phys_4d.h5"
with h5py.File(filename, 'r') as f:
    group = f['data']
    xint = group['xint'][:]
    t_points = group['t_points'][:]
    grid_d = group['grid'][:]
    tag = group['tag'][:]
    x1label = group['x1label'][:]
    x2label = group['x2label'][:]

In [9]:
x1label = "HA [kcal/mol]"
x2label = "Temperature [K]"
alabel = "Total product conc. [M]"
slabel = "Dominant product"
afilename = "HaT_activity_map.png"
sfilename = "HaT_selectivity_map.png"
product_id = ["HCOO[Si]", "CH$_2$(O[Si])$_2$", "CH$_3$(O[Si])"]
x1base = 40
x2base = 20


activity_grid = np.sum(grid_d, axis=0).T
selectivity_grid = np.argmax(grid_d, axis=0).T

from scipy.interpolate import RegularGridInterpolator
x = np.linspace(xint.min(), xint.max(), 200)
y = np.linspace(t_points.min(), t_points.max(), 200)
xg, yg = np.meshgrid(x, y, indexing='ij')
interp1 = RegularGridInterpolator((x, y), activity_grid, method="slinear")
interp2 = RegularGridInterpolator((x, y), selectivity_grid, method="slinear")

x = np.linspace(xint.min(), xint.max(), 2500)
y = np.linspace(t_points.min(), t_points.max(), 2500)
xn, yn = np.meshgrid(x, y, indexing='ij')
activity_grid_n = interp1((xn, yn))
selectivity_grid_n = interp2((xn, yn))

#plt.axhline(y = 273.15, xmin = -40, xmax = 40, color = 'black', linestyle = 'dashed') 
#plt.axhline(y = 373.15, xmin = -40, xmax = 40, color = 'black', linestyle = 'dashed') 

In [10]:
temp_lb = 260
temp_ub = 400
plot_3d_np(
    x,
    y,
    activity_grid_n,
    activity_grid.min(),
    1,
    -100,
    20,
    temp_lb,
    temp_ub,
    x1base=x1base,
    x2base=x2base,
    x1label=x1label,
    x2label=x2label,
    ylabel=alabel,
    filename=afilename,
    cmap="jet",
)

plot_3d_contour_regions_np(
    x,
    y,
    selectivity_grid_n,
    -100,
    20,
    temp_lb,
    temp_ub,
    x1base=x1base,
    x2base=x2base,
    x1label=x1label,
    x2label=x2label,
    ylabel=slabel,
    filename=sfilename,
    id_labels=product_id,
    nunique=len(product_id),
)

/tmp/ipykernel_6259/3932567192.py:109: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap=cm.get_cmap(cmap, len(levels)),


now for the final panel (CO2:Si-H = 20:5)

In [18]:
# read h5 files 
filename = "data/pincer_co2/mkm_descr_phys_4g.h5"
with h5py.File(filename, 'r') as f:
    group = f['data']
    xint = group['xint'][:]
    t_points = group['t_points'][:]
    grid_d = group['grid'][:]
    tag = group['tag'][:]
    x1label = group['x1label'][:]
    x2label = group['x2label'][:]

In [20]:
x1label = "HA [kcal/mol]"
x2label = "Temperature [K]"
alabel = "Total product conc. [M]"
slabel = "Dominant product"
afilename = "HaT_activity_map.png"
sfilename = "HaT_selectivity_map.png"
product_id = ["HCOO[Si]", "CH$_2$(O[Si])$_2$", "CH$_3$(O[Si])"]
x1base = 40
x2base = 20


activity_grid = np.sum(grid_d, axis=0).T
selectivity_grid = np.argmax(grid_d, axis=0).T

from scipy.interpolate import RegularGridInterpolator
x = np.linspace(xint.min(), xint.max(), 250)
y = np.linspace(t_points.min(), t_points.max(), 250)
xg, yg = np.meshgrid(x, y, indexing='ij')
interp1 = RegularGridInterpolator((x, y), activity_grid, method="slinear")
interp2 = RegularGridInterpolator((x, y), selectivity_grid, method="slinear")

x = np.linspace(xint.min(), xint.max(), 2500)
y = np.linspace(t_points.min(), t_points.max(), 2500)
xn, yn = np.meshgrid(x, y, indexing='ij')
activity_grid_n = interp1((xn, yn))
selectivity_grid_n = interp2((xn, yn))

#plt.axhline(y = 273.15, xmin = -40, xmax = 40, color = 'black', linestyle = 'dashed') 
#plt.axhline(y = 373.15, xmin = -40, xmax = 40, color = 'black', linestyle = 'dashed') 

In [21]:
temp_lb = 260
temp_ub = 400
plot_3d_np(
    x,
    y,
    activity_grid_n,
    activity_grid.min(),
    1,
    -100,
    20,
    temp_lb,
    temp_ub,
    x1base=x1base,
    x2base=x2base,
    x1label=x1label,
    x2label=x2label,
    ylabel=alabel,
    filename=afilename,
    cmap="jet",
)

plot_3d_contour_regions_np(
    x,
    y,
    selectivity_grid_n,
    -100,
    20,
    temp_lb,
    temp_ub,
    x1base=x1base,
    x2base=x2base,
    x1label=x1label,
    x2label=x2label,
    ylabel=slabel,
    filename=sfilename,
    id_labels=product_id,
    nunique=len(product_id),
)

/tmp/ipykernel_6259/3932567192.py:109: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap=cm.get_cmap(cmap, len(levels)),


- 2D volcano plots

In [22]:
def plot_2d_combo(
    x,
    y,
    px,
    py,
    ci=None,
    xmin=0,
    xmax=100,
    xbase=20,
    ybase=10,
    xlabel="X-axis",
    ylabel="Y-axis",
    filename="plot.png",
    rid=None,
    ms=None,
    rb=None,
    plotmode=1,
    labels=None,
    figsize=[7.5, 4],
):
    COLORS = [
        "#FF6347",
        "#009051",
        "#4169E1",
        "#9607BC",
        "#EA2CD3",
        "#D88918",
        "#148873",
        "#000000",
    ]

    marker = ["o", "^", "s", "P", "*", "X", "d", "D"]
    fig, ax = plt.subplots(
        frameon=False, figsize=figsize, dpi=300, constrained_layout=True
    )
    # Labels and key
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(xmin, xmax)
    plt.xticks(np.arange(xmin, xmax + 0.1, xbase))

    # no scatter plot
    if plotmode == 0:
        for i, yi in enumerate(y):
            ax.plot(
                x, yi, "-", linewidth=2.2, color=COLORS[i], alpha=0.95, label=labels[i]
            )
            ax = beautify_ax(ax)
            if rid is not None and rb is not None:
                avgs = []
                rb.append(xmax)
                for i in range(len(rb) - 1):
                    avgs.append((rb[i] + rb[i + 1]) / 2)
                for i in rb:
                    ax.axvline(
                        i, linestyle="dashed", color="black", linewidth=2, alpha=0.75
                    )

    # mono color scatter plot
    elif plotmode > 0:
        for i, yi in enumerate(y):
            ax.plot(
                x,
                yi,
                "-",
                linewidth=2,
                color=COLORS[i],
                alpha=0.95,
                zorder=1,
                label=labels[i],
            )
            ax = beautify_ax(ax)
            if rid is not None and rb is not None:
                avgs = []
                rb.append(xmax)
                for i in range(len(rb) - 1):
                    avgs.append((rb[i] + rb[i + 1]) / 2)
                for i in rb:
                    ax.axvline(
                        i,
                        linestyle="dashed",
                        color="black",
                        linewidth=2,
                        alpha=0.75,
                        zorder=3,
                    )
            if ci[i] is not None:
                plot_ci(ci[i], x, y[i], ax=ax)
            plotpoints(ax, px, py[i], np.repeat([COLORS[i]], len(px)), ms, plotmode)

    ymin, ymax = ax.get_ylim()
    ymax = bround(ymax, ybase, type="max")
    ymin = bround(ymin, ybase, type="min")
    plt.ylim(0, 1.05)
    plt.yticks(np.arange(0, 1.1, ybase))
    plt.legend(fontsize=16, loc="upper left", frameon=False, borderpad=0, handlelength=1)


#  fig.savefig(filename, dpi=400)
#

In [23]:

filename = "data/pincer_co2/vp/data_a.h5"
# read h5 files 
with h5py.File(filename, 'r') as f:
    # access the group containing the datasets
    group = f['data']
    # load each dataset into a numpy array
    descr_all = group['descr_all'][:]
    prod_conc_ = group['prod_conc_'][:]
    descrp_pt = group['descrp_pt'][:]
    prod_conc_pt_ = group['prod_conc_pt_'][:]
    cb = group['cb'][:]
    ms = group['ms'][:]
    cb = np.char.decode(cb)
    ms = np.char.decode(ms)
    tag = group['tag'][:]
    xlabel = group['xlabel'][:]
    ylabel = group['ylabel'][:]

import scipy as sp   
x =  descr_all 
y1 = prod_conc_[0]
y2 = prod_conc_[1]
y3 = prod_conc_[2]

# We use interp1d to interpolate to an arbitrary number of points
ip1 = sp.interpolate.interp1d(x, y1, kind='slinear')
ip2 = sp.interpolate.interp1d(x, y2, kind='slinear')
ip3 = sp.interpolate.interp1d(x, y3, kind='slinear')
x = np.linspace(-100, 20, 5000)
y1 = ip1(x)
y2 = ip2(x)
y3 = ip3(x)
ys = [y1, y2, y3]

xlabel = "HA [kcal/mol]"
ylabel = ylabel[0].decode()
tag = tag[0].decode()

if np.any(np.max(prod_conc_pt_) > 10):
    print("Concentration likely reported as %yield, set y_base to 10")
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8)
    if ybase == 0: ybase = 5
    ylabel = "%yield"
else:
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8, 1)
    if ybase == 0: ybase = 0.05
    ylabel = "Final product conc. [M]"

xbase = np.round((np.max(descr_all) - np.min(descr_all)) / 8)
if xbase == 0: xbase = 5

product_id = ['HCOO[Si]', 'CH$_2$(O[Si])$_2$', 'CH$_3$(O[Si])']

#NOTE change y_base to 0.2 for 5a and 5c
plot_2d_combo(
    x,
    ys,
    descrp_pt,
    prod_conc_pt_,
    # xmin=descr_all[0],
    # xmax=descr_all[-1],
    xmin=-100,
    xmax=20,
    ms=ms,
    xbase=20,
    ybase=0.2,
    xlabel=xlabel,
    ylabel=ylabel,
    filename=f"km_volcano_{tag}_combo_polished.png",
    plotmode=0,
    labels=product_id,
    ci=[None]*3)

plt.axvline(x = -32, color = 'black', linestyle = 'dashed') 
plt.axvline(x = -42, color = 'black', linestyle = 'dashed') 

In [26]:
def plot_2d_combo(
    x,
    y,
    px,
    py,
    ci=None,
    xmin=0,
    xmax=100,
    xbase=20,
    ybase=10,
    xlabel="X-axis",
    ylabel="Y-axis",
    filename="plot.png",
    rid=None,
    ms=None,
    rb=None,
    plotmode=1,
    labels=None,
    figsize=[7.5, 4],
):
    COLORS = [
        "#FF6347",
        "#009051",
        "#4169E1",
        "#9607BC",
        "#EA2CD3",
        "#D88918",
        "#148873",
        "#000000",
    ]

    marker = ["o", "^", "s", "P", "*", "X", "d", "D"]
    fig, ax = plt.subplots(
        frameon=False, figsize=figsize, dpi=300, constrained_layout=True
    )
    # Labels and key
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(xmin, xmax)
    plt.xticks(np.arange(xmin, xmax + 0.1, xbase))

    # no scatter plot
    if plotmode == 0:
        for i, yi in enumerate(y):
            ax.plot(
                x, yi, "-", linewidth=2.2, color=COLORS[i], alpha=0.95, label=labels[i]
            )
            ax = beautify_ax(ax)
            if rid is not None and rb is not None:
                avgs = []
                rb.append(xmax)
                for i in range(len(rb) - 1):
                    avgs.append((rb[i] + rb[i + 1]) / 2)
                for i in rb:
                    ax.axvline(
                        i, linestyle="dashed", color="black", linewidth=2, alpha=0.75
                    )

    # mono color scatter plot
    elif plotmode > 0:
        for i, yi in enumerate(y):
            ax.plot(
                x,
                yi,
                "-",
                linewidth=2,
                color=COLORS[i],
                alpha=0.95,
                zorder=1,
                label=labels[i],
            )
            ax = beautify_ax(ax)
            if rid is not None and rb is not None:
                avgs = []
                rb.append(xmax)
                for i in range(len(rb) - 1):
                    avgs.append((rb[i] + rb[i + 1]) / 2)
                for i in rb:
                    ax.axvline(
                        i,
                        linestyle="dashed",
                        color="black",
                        linewidth=2,
                        alpha=0.75,
                        zorder=3,
                    )
            if ci[i] is not None:
                plot_ci(ci[i], x, y[i], ax=ax)
            plotpoints(ax, px, py[i], np.repeat([COLORS[i]], len(px)), ms, plotmode)

    ymin, ymax = ax.get_ylim()
    ymax = bround(ymax, ybase, type="max")
    ymin = bround(ymin, ybase, type="min")
    plt.ylim(0, 3.05)
    plt.yticks(np.arange(0, 3.1, ybase))
    plt.legend(fontsize=16, loc="upper left", frameon=False, borderpad=0, handlelength=1)


    fig.savefig(filename, dpi=400)
#

In [27]:

filename = "data/pincer_co2/vp/data_b.h5"
# read h5 files 
with h5py.File(filename, 'r') as f:
    # access the group containing the datasets
    group = f['data']
    # load each dataset into a numpy array
    descr_all = group['descr_all'][:]
    prod_conc_ = group['prod_conc_'][:]
    descrp_pt = group['descrp_pt'][:]
    prod_conc_pt_ = group['prod_conc_pt_'][:]
    cb = group['cb'][:]
    ms = group['ms'][:]
    cb = np.char.decode(cb)
    ms = np.char.decode(ms)
    tag = group['tag'][:]
    xlabel = group['xlabel'][:]
    ylabel = group['ylabel'][:]

import scipy as sp   
x =  descr_all 
y1 = prod_conc_[0]
y2 = prod_conc_[1]
y3 = prod_conc_[2]

# We use interp1d to interpolate to an arbitrary number of points
ip1 = sp.interpolate.interp1d(x, y1, kind='slinear')
ip2 = sp.interpolate.interp1d(x, y2, kind='slinear')
ip3 = sp.interpolate.interp1d(x, y3, kind='slinear')
x = np.linspace(-100, 20, 5000)
y1 = ip1(x)
y2 = ip2(x)
y3 = ip3(x)
ys = [y1, y2, y3]

xlabel = "HA [kcal/mol]"
ylabel = ylabel[0].decode()
tag = tag[0].decode()

if np.any(np.max(prod_conc_pt_) > 10):
    print("Concentration likely reported as %yield, set y_base to 10")
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8)
    if ybase == 0: ybase = 5
    ylabel = "%yield"
else:
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8, 1)
    if ybase == 0: ybase = 0.05
    ylabel = "Final product conc. [M]"

xbase = np.round((np.max(descr_all) - np.min(descr_all)) / 8)
if xbase == 0: xbase = 5

product_id = ['HCOO[Si]', 'CH$_2$(O[Si])$_2$', 'CH$_3$(O[Si])']

#NOTE change y_base to 0.2 for 5a and 5c
plot_2d_combo(
    x,
    ys,
    descrp_pt,
    prod_conc_pt_,
    # xmin=descr_all[0],
    # xmax=descr_all[-1],
    xmin=-100,
    xmax=20,
    ms=ms,
    xbase=20,
    ybase=0.5,
    xlabel=xlabel,
    ylabel=ylabel,
    filename=f"km_volcano_{tag}_combo_polished.png",
    plotmode=0,
    labels=product_id,
    ci=[None]*3)

plt.axvline(x = -32, color = 'black', linestyle = 'dashed') 
plt.axvline(x = -42, color = 'black', linestyle = 'dashed') 

In [29]:

filename = "data/pincer_co2/vp/data_c.h5"
# read h5 files 
with h5py.File(filename, 'r') as f:
    # access the group containing the datasets
    group = f['data']
    # load each dataset into a numpy array
    descr_all = group['descr_all'][:]
    prod_conc_ = group['prod_conc_'][:]
    descrp_pt = group['descrp_pt'][:]
    prod_conc_pt_ = group['prod_conc_pt_'][:]
    cb = group['cb'][:]
    ms = group['ms'][:]
    cb = np.char.decode(cb)
    ms = np.char.decode(ms)
    tag = group['tag'][:]
    xlabel = group['xlabel'][:]
    ylabel = group['ylabel'][:]

import scipy as sp   
x =  descr_all 
y1 = prod_conc_[0]
y2 = prod_conc_[1]
y3 = prod_conc_[2]

# We use interp1d to interpolate to an arbitrary number of points
ip1 = sp.interpolate.interp1d(x, y1, kind='quadratic')
ip2 = sp.interpolate.interp1d(x, y2, kind='quadratic')
ip3 = sp.interpolate.interp1d(x, y3, kind='quadratic')
x = np.linspace(-100, 20, 5000)
y1 = ip1(x)
y2 = ip2(x)
y3 = ip3(x)
ys = [y1, y2, y3]

xlabel = "HA [kcal/mol]"
ylabel = ylabel[0].decode()
tag = tag[0].decode()

if np.any(np.max(prod_conc_pt_) > 10):
    print("Concentration likely reported as %yield, set y_base to 10")
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8)
    if ybase == 0: ybase = 5
    ylabel = "%yield"
else:
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8, 1)
    if ybase == 0: ybase = 0.05
    ylabel = "Final product conc. [M]"

xbase = np.round((np.max(descr_all) - np.min(descr_all)) / 8)
if xbase == 0: xbase = 5

product_id = ['HCOO[Si]', 'CH$_2$(O[Si])$_2$', 'CH$_3$(O[Si])']

#NOTE change y_base to 0.2 for 5a and 5c
plot_2d_combo(
    x,
    ys,
    descrp_pt,
    prod_conc_pt_,
    # xmin=descr_all[0],
    # xmax=descr_all[-1],
    xmin=-100,
    xmax=20,
    ms=ms,
    xbase=20,
    ybase=0.5,
    xlabel=xlabel,
    ylabel=ylabel,
    filename=f"km_volcano_{tag}_combo_polished.png",
    plotmode=0,
    labels=product_id,
    ci=[None]*3)

plt.axvline(x = -32, color = 'black', linestyle = 'dashed') 
plt.axvline(x = -42, color = 'black', linestyle = 'dashed') 

In [32]:

filename = "data/pincer_co2/vp/data_d.h5"
# read h5 files 
with h5py.File(filename, 'r') as f:
    # access the group containing the datasets
    group = f['data']
    # load each dataset into a numpy array
    descr_all = group['descr_all'][:]
    prod_conc_ = group['prod_conc_'][:]
    descrp_pt = group['descrp_pt'][:]
    prod_conc_pt_ = group['prod_conc_pt_'][:]
    cb = group['cb'][:]
    ms = group['ms'][:]
    cb = np.char.decode(cb)
    ms = np.char.decode(ms)
    tag = group['tag'][:]
    xlabel = group['xlabel'][:]
    ylabel = group['ylabel'][:]

import scipy as sp   
x =  descr_all 
y1 = prod_conc_[0]
y2 = prod_conc_[1]
y3 = prod_conc_[2]

# We use interp1d to interpolate to an arbitrary number of points
ip1 = sp.interpolate.interp1d(x, y1, kind='quadratic')
ip2 = sp.interpolate.interp1d(x, y2, kind='quadratic')
ip3 = sp.interpolate.interp1d(x, y3, kind='quadratic')
x = np.linspace(-100, 20, 5000)
y1 = ip1(x)
y2 = ip2(x)
y3 = ip3(x)
ys = [y1, y2, y3]

xlabel = "HA [kcal/mol]"
ylabel = ylabel[0].decode()
tag = tag[0].decode()

if np.any(np.max(prod_conc_pt_) > 10):
    print("Concentration likely reported as %yield, set y_base to 10")
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8)
    if ybase == 0: ybase = 5
    ylabel = "%yield"
else:
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8, 1)
    if ybase == 0: ybase = 0.05
    ylabel = "Final product conc. [M]"

xbase = np.round((np.max(descr_all) - np.min(descr_all)) / 8)
if xbase == 0: xbase = 5

product_id = ['HCOO[Si]', 'CH$_2$(O[Si])$_2$', 'CH$_3$(O[Si])']

#NOTE change y_base to 0.2 for 5a and 5c
plot_2d_combo(
    x,
    ys,
    descrp_pt,
    prod_conc_pt_,
    # xmin=descr_all[0],
    # xmax=descr_all[-1],
    xmin=-100,
    xmax=20,
    ms=ms,
    xbase=20,
    ybase=0.5,
    xlabel=xlabel,
    ylabel=ylabel,
    filename=f"km_volcano_{tag}_combo_polished.png",
    plotmode=0,
    labels=product_id,
    ci=[None]*3)

plt.axvline(x = -32, color = 'black', linestyle = 'dashed') 
plt.axvline(x = -42, color = 'black', linestyle = 'dashed') 

In [33]:

filename = "data/pincer_co2/vp/mkm_vp_298.h5"
# read h5 files 
with h5py.File(filename, 'r') as f:
    # access the group containing the datasets
    group = f['data']
    # load each dataset into a numpy array
    descr_all = group['descr_all'][:]
    prod_conc_ = group['prod_conc_'][:]
    descrp_pt = group['descrp_pt'][:]
    prod_conc_pt_ = group['prod_conc_pt_'][:]
    cb = group['cb'][:]
    ms = group['ms'][:]
    cb = np.char.decode(cb)
    ms = np.char.decode(ms)
    tag = group['tag'][:]
    xlabel = group['xlabel'][:]
    ylabel = group['ylabel'][:]

import scipy as sp   
x =  descr_all 
y1 = prod_conc_[0]
y2 = prod_conc_[1]
y3 = prod_conc_[2]

# We use interp1d to interpolate to an arbitrary number of points
ip1 = sp.interpolate.interp1d(x, y1, kind='quadratic')
ip2 = sp.interpolate.interp1d(x, y2, kind='quadratic')
ip3 = sp.interpolate.interp1d(x, y3, kind='quadratic')
x = np.linspace(-100, 20, 5000)
y1 = ip1(x)
y2 = ip2(x)
y3 = ip3(x)
ys = [y1, y2, y3]

xlabel = "HA [kcal/mol]"
ylabel = ylabel[0].decode()
tag = tag[0].decode()

if np.any(np.max(prod_conc_pt_) > 10):
    print("Concentration likely reported as %yield, set y_base to 10")
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8)
    if ybase == 0: ybase = 5
    ylabel = "%yield"
else:
    ybase = np.round((np.max(prod_conc_pt_) - 0) / 8, 1)
    if ybase == 0: ybase = 0.05
    ylabel = "Final product conc. [M]"

xbase = np.round((np.max(descr_all) - np.min(descr_all)) / 8)
if xbase == 0: xbase = 5

product_id = ['HCOO[Si]', 'CH$_2$(O[Si])$_2$', 'CH$_3$(O[Si])']

#NOTE change y_base to 0.2 for 5a and 5c
plot_2d_combo(
    x,
    ys,
    descrp_pt,
    prod_conc_pt_,
    # xmin=descr_all[0],
    # xmax=descr_all[-1],
    xmin=-100,
    xmax=20,
    ms=ms,
    xbase=20,
    ybase=0.5,
    xlabel=xlabel,
    ylabel=ylabel,
    filename=f"km_volcano_{tag}_combo_polished.png",
    plotmode=0,
    labels=product_id,
    ci=[None]*3)

plt.axvline(x = -32, color = 'black', linestyle = 'dashed') 
plt.axvline(x = -42, color = 'black', linestyle = 'dashed') 